In [84]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
%autoreload

import pandas as pd
import utilities as utils
import catboost as cb
import numpy as np

from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.model_selection import train_test_split

In [86]:
data_list = utils.read_data()
Data = utils.DataProcessor(data_list)
df = Data.create_train_data() # NaNs are filled with 0
df.sort_index(inplace=True)
df = df.loc['2019-09-04 08:00:00':]

/home/stinky/Documents/ntnu/ml/TDT4173-ML/utilities.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(0, inplace=True)
/home/stinky/Documents/ntnu/ml/TDT4173-ML/utilities.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.fillna(0, inplace=True)


In [87]:
# Check absent values
null_value_stats = df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

In [88]:
df.fillna(-9999, inplace=True)

In [89]:
X = df.drop('pv_measurement', axis=1)
y = df['pv_measurement']

In [90]:
print(X.dtypes)
categorical_features_indices = np.where(X.dtypes != float)[0]

absolute_humidity_2m:gm3          float32
air_density_2m:kgm3               float32
ceiling_height_agl:m              float32
clear_sky_energy_1h:J             float32
clear_sky_rad:W                   float32
cloud_base_agl:m                  float32
dew_or_rime:idx                   float32
dew_point_2m:K                    float32
diffuse_rad:W                     float32
diffuse_rad_1h:J                  float32
direct_rad:W                      float32
direct_rad_1h:J                   float32
effective_cloud_cover:p           float32
elevation:m                       float32
fresh_snow_12h:cm                 float32
fresh_snow_1h:cm                  float32
fresh_snow_24h:cm                 float32
fresh_snow_3h:cm                  float32
fresh_snow_6h:cm                  float32
is_day:idx                        float32
is_in_shadow:idx                  float32
msl_pressure:hPa                  float32
precip_5min:mm                    float32
precip_type_5min:idx              

In [91]:
# Set X_train to 75% of df and X_validation to 25% without train_test_split
X_train = X.iloc[:int(len(X)*0.75)]
X_validation = X.iloc[int(len(X)*0.75):]
y_train = y.iloc[:int(len(y)*0.75)]
y_validation = y.iloc[int(len(y)*0.75):]

X_test = Data.create_test_data()

y_validation.head()

date_forecast
2022-02-28 21:15:00    0.0
2022-02-28 21:30:00    0.0
2022-02-28 21:30:00    0.0
2022-02-28 21:30:00    0.0
2022-02-28 21:45:00    0.0
Name: pv_measurement, dtype: float64

In [92]:
model = CatBoostClassifier(
            custom_loss=[metrics.Accuracy()],
            random_seed=42,
            logging_level='Silent',
)

In [93]:
model.fit(
    X_train, y_train,
    # cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation)
    # plot=True
);

CatBoostError: /src/catboost/catboost/private/libs/algo/data.cpp:196: Dataset test #0 contains class label "2372.7" that is not present in the learn dataset